In [1]:
import numpy as np
import pandas as pd

C:\Users\Sonat\AppData\Local\Temp\ipykernel_11752\1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def power_iteration(A, max_iter=100, tol=1e-6):
    """
    Power Iteration method to find the dominant singular vector of matrix A.

    Parameters:
        A (numpy.ndarray): Input matrix.
        max_iter (int): Maximum number of iterations (default: 100).
        tol (float): Tolerance for convergence (default: 1e-6).

    Returns:
        eig_val (float): The greatest (in absolute value) eigenvalue of A.
        v (numpy.ndarray): Nonzero vector which is a corresponding eigenvector of eig_val.
    """
    m, n = A.shape
    v = np.random.rand(n, 1)

    for _ in range(max_iter):
        Av = np.dot(A, v)
        norm_av = np.linalg.norm(Av)
        v_new = Av / norm_av

        # Check for convergence
        if np.linalg.norm(v_new - v) < tol:
            break

        v = v_new

    # Compute the corresponding singular value
    eig_val = np.dot(np.dot(A.T, A.dot(v)), v) / np.dot(v.T, v)

    return eig_val, v

In [3]:
def svd_with_deflation(A, num_singular_values=1, max_iter=100, tol=1e-6):
    """
    Perform Singular Value Decomposition (SVD) using power iteration with deflation.

    Parameters:
    - A (numpy.ndarray): The input matrix for which the SVD will be computed.
    - num_singular_values (int): Number of singular values to compute (default: 1).
    - max_iter (int): Maximum number of iterations for power iteration (default: 100).
    - tol (float): Tolerance for convergence in power iteration (default: 1e-6).

    Returns:
    - tuple: A tuple containing the matrices U, Sigma, and V^T, representing the SVD of `A`.
    """
    
    U = []  # List to store left singular vectors
    Sigma = []  # List to store singular values
    Vt = []  # List to store transpose of right singular vectors

    for _ in range(num_singular_values):
        # Use power iteration to find the dominant singular vector and value
        singular_value, singular_vector = power_iteration(A, max_iter, tol)

        # Store the computed singular vectors and values
        Sigma.append(singular_value)
        Vt.append(singular_vector)

        # Deflation: Subtract the contribution of the computed singular vector and value
        outer_product = singular_value * np.outer(singular_vector, singular_vector)
        A = A - outer_product

    # Sort singular values and corresponding vectors in descending order
    sorted_indices = np.argsort(Sigma)[::-1]
    Sigma = np.array(Sigma)[sorted_indices]
    Vt = np.array(Vt)[sorted_indices]

    # Calculate singular values
    Sigma = np.sqrt(Sigma)

    # Assemble U, Sigma, and V^T
    Vt = Vt.T
    U = A.dot(Vt) / Sigma

    return U, Sigma, Vt
